In [60]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime

def get_match_data(start_date,time_now_formatted,devengine):


    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    #Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,)

    #Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,)

    return elo_hard, elo_clay, elo_data_hard, elo_data_clay

# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today =time_now
two_years_ago = (today - datetime.timedelta(days=365*2)).strftime("%Y-%m-%d")

elo_hard, elo_clay, elo_data_hard, elo_data_clay = get_match_data(two_years_ago,time_now_formatted,devengine)

def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0

def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)

        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                    "Resulted":[row.Resulted],
                    "fav_rank":[row.Fav_Rank],
                    "dog_rank":[row.Dog_Rank],
                    "Elo_Fav_Elo":[row.Elo_Fav_Elo],
                    "Elo_Dog_Elo":[row.Elo_Dog_Elo],                    
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df

results_hard=get_filtered_data(elo_data_hard,elo_hard)
results_clay=get_filtered_data(elo_data_clay,elo_clay)
def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_csv('mensserving.csv')

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_csv('womensserving.csv')

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        #serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(result_df, serving, how='left', left_on=['Fav'], right_on=['Name'])
            result = pd.merge(result,serving, how='left', left_on=['Dog'], right_on=['Name'])
            result.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,)
            result.drop(columns=['Name_x','Name_y'],inplace=True)
    

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None
        
    return result, result_serving


_,serving_hard=process_serving_data(results_hard)
_,serving_clay=process_serving_data(results_clay)

elo_hardxx = pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2022-01-01' and Date not like '{time_now_formatted}' --and Fav_odds>1.2 and Fav_odds<1.4 ",
    con=devengine,)

data_concat = pd.DataFrame(columns=['Date', 'Player', 'Odds', 'Win/Loss'])
for dataset_type in [('Winner', 'Win'), ('Loser', 'Loss')]:
    df = elo_hard[['Date', dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df['Player'] = df[dataset_type[0]]
    df['Odds'] = df[f"{dataset_type[0]}_Odds"]
    df['Win/Loss'] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat['Odds'] = data_concat.Odds.astype(float)

def analyse_matchups(result_df,data_concat):
    for _,matchup in result_df.iterrows():
        player1=matchup.Fav
        player2=matchup.Dog
        player1_odds=float(matchup.Fav_Odds)
        player1_odds_hi=player1_odds+0.15
        player1_odds_lo=player1_odds-0.15
        player2_odds=float(matchup.Dog_Odds)
        player2_odds_hi=player2_odds+0.15
        player2_odds_lo=player2_odds-0.15
        player1=data_concat[data_concat['Player']==player1].copy()
        player2=data_concat[data_concat['Player']==player2].copy()
        player2=player2[(player2['Odds']>player2_odds_lo)&(player2['Odds']<player2_odds_hi)]
        if len(player2)>0:
            winperc2=len(player2[player2['Win/Loss']=='Win'])/len(player2)
        else:
            winperc2=0
        player1=player1[(player1['Odds']>player1_odds_lo)&(player1['Odds']<player1_odds_hi)]
        if len(player1)>0:
            winperc1=len(player1[player1['Win/Loss']=='Win'])/len(player1)
        else:
            winperc1=0
        if len(player1)>5 and len(player2)>3:
            print(matchup.Time,f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})")
            print(len(player1),winperc1,len(player2),winperc2)


analyse_matchups(serving_clay,data_concat)
final_hard=serving_hard
final_clay=serving_clay

00:05 Veronika Kudermetova (1.72-->2.02) Anastasia Potapova (1.8-->2.1)
6 0.5 6 0.8333333333333334
19:00 Antoine Escoffier (1.56-->1.86) Matteo Martineau (1.84-->2.14)
23 0.6956521739130435 4 0.5
19:00 Alexei Popyrin (1.61-->1.91) Roman Safiullin (1.95-->2.25)
10 0.4 7 0.8571428571428571
19:00 Marketa Vondrousova (1.43-->1.73) Maria Sakkari (2.26-->2.56)
12 0.5833333333333334 4 0.5
19:00 Ulises Blanch (1.48-->1.78) Alibek Kachmazov (1.95-->2.25)
9 0.7777777777777778 5 0.6
20:30 Shintaro Mochizuki (1.38-->1.68) Andrea Arnaboldi (2.15-->2.45)
14 0.7857142857142857 9 0.4444444444444444
22:00 Frances Tiafoe (1.38-->1.68) Daniel Altmaier (2.38-->2.68)
10 0.9 4 0.0


In [61]:
def last_five(df,pastmatches):
    for index, row in df.iterrows():
        fav=row.Fav
        dog=row.Dog
        last_five_matches_fav=pastmatches[(pastmatches['Winner']==fav)|(pastmatches['Loser']==fav)].tail(5)
        if len(last_five_matches_fav)>0:  
            fav_last_five_win_perc=len(last_five_matches_fav[last_five_matches_fav['Winner']==fav])/len(last_five_matches_fav)
        else:
            fav_last_five_win_perc=0
        last_five_matches_dog=pastmatches[(pastmatches['Winner']==dog)|(pastmatches['Loser']==dog)].tail(5)
        if len(last_five_matches_dog)>0:    
            dog_last_five_win_perc=len(last_five_matches_dog[last_five_matches_dog['Winner']==dog])/len(last_five_matches_dog) 
        else:
            dog_last_five_win_perc=0
        df.at[index, 'fav_last_five_win_perc'] = fav_last_five_win_perc
        df.at[index, 'dog_last_five_win_perc'] = dog_last_five_win_perc
    return df

final_hard=last_five(final_hard,elo_hard)
#x=x[x['Resulted']=='False']
#x[['Fav','Dog','fav_rank','dog_rank','Fav_Odds','Dog_Odds','fav_last_five_win_perc','dog_last_five_win_perc','Time']].sort_values(by='Time')
final_hard

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,fav_rank,dog_rank,Elo_Fav_Elo,Elo_Dog_Elo,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc
0,13:00,1.83,1.92,Max Purcell,Max Purcell,62%,21,Aleksandar Vukic,52%,21,...,87.0,127.0,1886.165349,1830.016204,0.68,0.2,0.85,0.13,0.8,0.8


In [62]:
final_clay=last_five(final_clay,elo_clay)
#y=y[y['Resulted']=='False']
#y[['Fav','Dog','Fav_Rank','Dog_Rank','Fav_Odds','Dog_Odds','fav_last_five_win_perc','dog_last_five_win_perc','Time','Resulted']].sort_values(by='Time')


In [63]:
if len(final_hard)>0:
    final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)
    final_hard['Dog_Odds']=final_hard['Dog_Odds'].astype(float)

In [64]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Aleksandar Vukic beat Max Purcell


In [65]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Alessandro Giannessi beat Alberto Barroso Campos
Anastasia Potapova beat Veronika Kudermetova
Veronika Kudermetova beat Anastasia Potapova
Iga Swiatek beat Lesia Tsurenko
Cameron Norrie beat Marton Fucsovics


In [66]:
final_hard[(final_hard['Sex']!='Womens')&(final_hard['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,13:00,Max Purcell,Max Purcell,1.83,0.619048,87.0,0.68,0.2,0.8,Aleksandar Vukic,1.92,0.52381,127.0,0.85,0.13,0.8


In [67]:
final_clay['Dog_Odds']=final_clay['Dog_Odds'].astype(float)

In [68]:
final_hard
final_clay[(final_clay['Sex']=='Womens')&(final_clay['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
12,19:00,Marketa Vondrousova,Maria Sakkari,1.58,0.550000,70.0,0.73,0.34,0.8,Maria Sakkari,2.41,0.677419,8.0,0.75,0.33,0.8
13,19:00,Daria Kasatkina,Daria Kasatkina,1.15,0.666667,9.0,0.61,0.47,0.6,Julia Grabher,5.73,0.100000,89.0,0.64,0.40,0.6
19,19:00,Karolina Muchova,Karolina Muchova,1.69,0.600000,52.0,0.71,0.25,0.6,Camila Giorgi,2.19,0.523810,37.0,0.65,0.34,0.6
28,20:30,Iga Swiatek,Iga Swiatek,1.02,0.906977,1.0,0.76,0.50,0.8,Lesia Tsurenko,17.13,0.333333,68.0,0.61,0.44,0.8
29,20:30,Cori Gauff,Cori Gauff,1.26,0.651163,5.0,0.71,0.35,0.6,Marie Bouzkova,3.88,0.266667,38.0,0.73,0.39,0.4
35,22:00,Paula Badosa,Paula Badosa,1.24,0.514286,35.0,0.68,0.36,0.8,Marta Kostyuk,4.12,0.486486,40.0,0.64,0.35,0.2
38,23:00,Madison Keys,Madison Keys,1.82,0.600000,23.0,0.74,0.31,0.8,Victoria Azarenka,1.99,0.516129,17.0,0.67,0.40,0.6


In [69]:
final_clay[(final_clay['Sex']=='Mens')&(final_clay['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
7,19:00,Dominik Koepfer,Dominik Koepfer,1.45,0.636364,149.0,0.70,0.21,0.8,Jakub Mensik,2.66,0.500000,333.0,0.00,0.00,0.80
8,19:00,Antoine Escoffier,Matteo Martineau,1.71,0.769231,199.0,0.00,0.00,0.0,Matteo Martineau,1.99,0.200000,374.0,0.00,0.00,0.20
9,19:00,Francisco Cerundolo,Francisco Cerundolo,1.33,0.615385,31.0,0.75,0.28,0.6,Gregoire Barrere,3.42,0.666667,63.0,0.78,0.15,0.60
10,19:00,Alexei Popyrin,Alexei Popyrin,1.76,0.363636,77.0,0.80,0.12,0.8,Roman Safiullin,2.10,0.600000,103.0,0.86,0.19,0.80
11,19:00,Roberto Bautista Agut,Marco Cecchinato,1.53,0.583333,25.0,0.82,0.27,0.4,Marco Cecchinato,2.52,0.200000,83.0,0.70,0.22,0.80
14,19:00,Ulises Blanch,Ulises Blanch,1.63,0.538462,354.0,0.00,0.00,0.2,Alibek Kachmazov,2.10,0.500000,331.0,0.00,0.00,0.00
15,19:00,Nicolas Mejia,Nicolas Mejia,1.25,0.384615,235.0,0.00,0.00,0.4,Yanki Erel,3.49,0.444444,382.0,0.00,0.00,0.00
16,19:00,Steven Diez,Steven Diez,1.58,0.300000,270.0,0.63,0.00,0.6,Ugo Blanchet,2.19,0.400000,336.0,0.00,0.00,0.20
17,19:00,Dane Sweeny,Kyrian Jacquet,1.71,0.454545,236.0,0.00,0.00,0.4,Kyrian Jacquet,1.99,0.100000,428.0,0.00,0.00,0.60
18,19:00,Mate Valkusz,Mate Valkusz,1.16,0.454545,279.0,0.00,0.00,0.6,Christian Langmo,4.32,0.727273,345.0,0.00,0.00,0.20


In [70]:
final_clay.to_pickle('Clay_Today')
final_hard.to_pickle('Hard_Today')